In [ ]:
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12),
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
import pandas as pd
import os

def load_police_data():
    csv_path = os.path.join(os.path.join("police_data"), "crimes_2012_to_2017.csv")
    return pd.read_csv(csv_path, index_col=0)

## Load Data and get basic info about shape and values

In [ ]:
police_data = load_police_data()
print(f'Shape of table: {police_data.shape}')
# Show Unique value count for each column
print(f'Unique values per column: \n{police_data.nunique()}')
police_data.tail(2)

## Format Data, drop/rename/add columns, convert to timeseries etc

In [ ]:
drop_attributes = ['Location',
                   'ID',
                   'Case Number',
                   'IUCR',
                   'Block',
                   'Ward',
                   'Primary Type',
                   'Beat',
                   'X Coordinate',
                   'Community Area',
                   'Year',
                   'Latitude',
                   'Longitude',
                   'Y Coordinate',
                   'Updated On',
                   'Description',
                   'Location Description']

police_data = police_data.drop(columns=drop_attributes)

# Rename columns
police_data.rename(columns={
    'Arrest': 'Arrest Status',
    'Domestic': 'Domestic Status'
}, inplace=True)

# Convert to timeseries
# A lot (X50) faster when we provide the format
police_data['Date'] = pd.to_datetime(
    police_data['Date'], format='%m/%d/%Y %I:%M:%S %p')

# Set datetime as index, will not remove duplicates (verify_integrity=True would remove duplicates)
police_data = police_data.set_index('Date')

# Generate temp columns for plotting distributions
police_data['Year'] = police_data.index.year
police_data['Month'] = police_data.index.month
police_data['Day of the Year'] = police_data.index.day_of_year

# Drop rows with null values
police_data= police_data.dropna()
print(f'Shape of table: {police_data.shape}')

# Show Unique value count for each column
print(f'Unique values per column: \n{police_data.nunique()}')


## Visualize Categorical Values

In [ ]:
# Print bar diagrams for important columns
columns_hist = [
    'District',
    'FBI Code',
    'Arrest Status',
    'Domestic Status',
    'Year',
    'Month',
]

for column in columns_hist:
    police_data[column].value_counts().plot(kind='bar')
    plt.title(f'Distribution of crimes per {column}')
    plt.xlabel(column)
    plt.ylabel('Crime count')
    plt.show()

drop_attributes = [
    'Year',
    'Month',
    'Day of the Year'
]

police_data = police_data.drop(columns=drop_attributes)


## Timeseries Visualisation

In [ ]:
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(20, 10), 'lines.marker':'o', 'lines.linewidth': 0.5})
# comment out to see all possible keys
# mpl.rcParams.keys()

# this helps with adding plot ticks on certain weekdays etc
import matplotlib.dates as mdates

In [ ]:
ax = police_data.resample('1W').count().plot(linestyle='None', legend=False)
ax.set_ylabel('Weekly Crime count')
ax.set_title('Weekly crime count across 5 years')

In [ ]:
ax = police_data.loc['2015':'2016'].resample('1d').count().plot(linestyle='-', legend=False)
ax.set_ylabel('Daily Crime count')
ax.set_title('Daily crime count over the years 2015-2016')

In [ ]:
ax = police_data.loc['2015/03':'2015/08'].resample('1d').count().plot(linestyle='-', legend=False)
ax.set_ylabel('Daily Crime count')
ax.set_title('Daily crime count over Spring Summer 2015')
# Set x-axis major ticks to weekly interval, on Mondays
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MONDAY))
# Format x-tick labels as 3-letter month name and day number
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'));

## Autoregressive Integrated Moving Average Model

- AR: Autoregression. A model that uses the dependent relationship between an observation and some number of lagged observations.
- I: Integrated. The use of differencing of raw observations (e.g. subtracting an observation from an observation at the previous time step) in order to make the time series
  stationary.
- MA: Moving Average. A model that uses the dependency between an observation and a residual error from a moving average model applied to lagged observations.


### Prepare Series

In [ ]:
# remove all columns
print(f'Shape of table: {police_data.shape}')
drop_attributes = [
    'Arrest Status',
    'Domestic Status',
    'District',
    'FBI Code'
]
police_data = police_data.drop(columns=drop_attributes)


In [ ]:
# Create period table, with one column for crime count
print(f'Shape of table before resampling: {police_data.shape}')

police_data['Crime Count'] = ""
# table with monthly period, skip 2017 data cause they are too little
series = police_data.loc[:'2016/12'].resample(rule='1m', kind='period').count()
print(f'Shape of period array: {series.shape}')
print(series.info())
series.tail(2)


### Get Autocorrelation

#### The parameters of the ARIMA model are defined as follows:

- **p**: The number of lag observations included in the model, also called the lag order.
- **d**: The number of times that the raw observations are differenced, also called the degree of differencing.
- **q**: The size of the moving average window, also called the order of moving average.

In [ ]:
from pandas.plotting import autocorrelation_plot
# This can help as choose the lag to be used in the Arima model, p value
autocorrelation_plot(series)

### First try of training the ARIMA model

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from pandas import DataFrame

# fit model
model = ARIMA(series, order=(5,1,0))
model_fit = model.fit()
# summary of fit model
print(model_fit.summary())
# line plot of residuals
residuals = DataFrame(model_fit.resid)
residuals.plot()
plt.show()
# density plot of residuals
residuals.plot(kind='kde')
plt.show()
# summary stats of residuals
print(residuals.describe())

### Split series in train and validation sets, then run a walk-forward train and validation - same hyperparameters as in previous step


In [ ]:
from sklearn.metrics import mean_squared_error


def walk_forward_train_evaluate_and_report(series, arima_order):
    # split into train and validation sets
    X = series.values
    size = int(len(X) * 0.66)
    train, validation = X[0:size], X[size:len(X)]
    history = [x for x in train]
    predictions = list()
    # walk-forward validation
    for t in range(len(validation)):
        model = ARIMA(history, order=arima_order)
        # print('Fitting on:')
        # print(history)
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = validation[t]
        history.append(obs)
        print('predicted=%f, expected=%f' % (yhat, obs))
    # evaluate forecasts
    mse = mean_squared_error(validation, predictions)
    print('validation MSE: %.3f' % mse)
    # plot forecasts against actual outcomes
    plt.plot(validation)
    plt.plot(predictions, color='red')
    plt.show()


walk_forward_train_evaluate_and_report(series, (5, 1, 0))


### ARIMA hyperparameters tuning with the Grid Search Method

In [ ]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    error = mean_squared_error(test, predictions)
    return error


In [ ]:
# supress errors noise
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p, d, q)
                try:
                    mse = evaluate_arima_model(dataset, order)
                    if mse < best_score:
                        best_score, best_cfg = mse, order
                    print('ARIMA%s MSE=%.3f' % (order, mse))
                except:
                    continue
    print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
    return best_cfg


In [ ]:
# Split training/validation set and final test set
train_size = int(len(series) * 0.6)
train_series, test_series = series[0:train_size], series[train_size:]

In [ ]:
# evaluate parameters
p_values = [8, 10, 12]
d_values = range(0, 3)
q_values = range(0, 3)
best_cfg = evaluate_models(train_series.values, p_values, d_values, q_values)


###  We found the best hyperparameter combination, we can now evaluate the predictions against unseen before values

In [ ]:
walk_forward_train_evaluate_and_report(test_series, best_cfg)